# A. Récupération des données  

"Nous avons récupéré les données historiques de NVIDIA via l'API Yahoo Finance, une source ouverte fiable pour les données boursières."

## A.1 - Choix des données 
"explication"
- choix de la base 
- choix du cours 
- choix des dates (strat + end)

## A.2 - Importation et traitement des données

requierments : "pip install yfinance"

In [ ]:
import yfinance as yf

ticker = "NVDA"
start_date = "2015-01-01"
end_date = "2024-12-31"

# TODO : edit le text en anglais vers FR   
# Download historical stock data
# We use the yfinance  API to scrape open market data from Yahoo Finance.
# This satisfies the requirement of using open data sources.
nvda_data = yf.download(ticker, start=start_date, end=end_date)

# Save original raw data as per requirements
nvda_data.to_csv("nvda_stock_data_raw.csv")
print("Raw dataset saved to nvda_stock_data_raw.csv")


nvda_data.head()
# Reset index to make Date a column
nvda_data.reset_index(inplace=True)

# Flatten MultiIndex columns created by yfinance
nvda_data.columns = ['_'.join(col).strip() if isinstance(col, tuple) and col[1] else col[0] for col in nvda_data.columns.values]
nvda_data.head()

# Sort data by date
nvda_data = nvda_data.sort_values(by='Date')
nvda_data.head()

ModuleNotFoundError: No module named 'yfinance'

On va voir comment sont les données 

In [ ]:
# TODO : commentaire -> faire 1 seul gro commentaire ici 
# Check column names and data types
# Rigorous cleaning involves checking for missing values and ensuring correct data types.
nvda_data.dtypes

# Check for missing values
nvda_data.isnull().sum()
# Basic info
nvda_data.info()
# Basic descriptive statistics
nvda_data.describe()

# Check correlation between features
nvda_data.corr()

# B. Analyse Descriptive
"description partie"
- méhode utilisée
- pourquoi utilisée 
"base à traduire"
In this section, we explore NVIDIA stock price trends, returns, and volatility over time.
Key features like daily returns, moving averages, and rolling volatility are calculated.
Visualization helps identify patterns, trends, and risk levels, guiding feature selection for modeling.

## B.1 - "daily pourcentage" + stats classiques
"explain overall"
+ RAJOUTER DES STATS DESC 'meme si basiques'


"base à traduire" 
-> cell. 1
This line calculates the daily percentage change in the closing price of NVIDIA stock.
It shows how much the stock price increased or decreased compared to the previous trading day.
Formula used is Daily_Return = (Today's Close - Yesterday's Close) / Yesterday's Close
This feature is commonly used in financial analysis to measure stock performance and volatility.

-> cell. 2
These lines calculate moving averages of NVIDIA's closing price over different time windows.
MA20  → Short-term trend (approximately 1 trading month)
MA50  → Medium-term trend
MA200 → Long-term trend (widely used to identify overall market direction)
A moving average smooths short-term price fluctuations and highlights underlying trends.
It is commonly used in financial analysis to detect trend direction,
support/resistance levels, and potential buy/sell signals.

This line calculates the 20-day rolling volatility of NVIDIA stock.
Volatility is measured as the standard deviation of daily returns over the last 20 trading days.
It represents how much the stock price fluctuates in the short term.
Higher volatility → higher risk and uncertainty
Lower volatility  → more stable price movement
Rolling window of 20 days is commonly used to approximate one trading month.
This feature is widely used in risk analysis, portfolio management,
and financial forecasting models.

-> cell. 3 
GRAPH INTERPRETATION:
The plot below visualizes the Price Trends along with Moving Averages.
The Close Price shows the actual market value.
SMA lines (20, 50, 200) help identify the prevailing trend direction.
Crossovers between Short-term (MA20) and Long-term (MA200) often signal trend reversals (Golden Cross / Death Cross).

In [ ]:

nvda_data['Daily_Return'] = nvda_data['Close_NVDA'].pct_change()

In [ ]:
nvda_data['MA20'] = nvda_data['Close_NVDA'].rolling(window=20).mean()
nvda_data['MA50'] = nvda_data['Close_NVDA'].rolling(window=50).mean()
nvda_data['MA200'] = nvda_data['Close_NVDA'].rolling(window=200).mean()

nvda_data['Volatility'] = nvda_data['Daily_Return'].rolling(window=20).std()

## B.2 - Graphiques 
"explain overall"
+ RAJOUTER des GRAPHIQUES meme si c'est nul 

"base à traduire"
-> cell. 1 
GRAPH INTERPRETATION:
The plot below visualizes the Price Trends along with Moving Averages.
The Close Price shows the actual market value.
SMA lines (20, 50, 200) help identify the prevailing trend direction.
Crossovers between Short-term (MA20) and Long-term (MA200) often signal trend reversals (Golden Cross / Death Cross).

-> cell2. 
Plot Distribution of Daily Returns
This histogram shows how often different daily return values occur.
It helps visualize stock performance fluctuations and identify periods of high or low volatility.
The KDE curve highlights the overall distribution trend of daily returns.

-> cell. 3
GRAPH INTERPRETATION
Volatility measures the dispersion of returns.
Peaks in the red line below indicate periods of high market uncertainty and risk.
This is crucial for risk management and modifying trading strategies during turbulent times.

-> cell. 4
Plot Feature Correlation Heatmap
This heatmap visualizes the correlation between key features like closing price,
daily returns, moving averages, and volatility.
Strong correlations indicate how features are related and help guide feature selection
for modeling and understanding multicollinearity.


In [ ]:
#cell1

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (14,6)

plt.figure(figsize=(16,6))
plt.plot(nvda_data['Date'], nvda_data['Close_NVDA'], label='Close Price', color='blue')
plt.plot(nvda_data['Date'], nvda_data['MA20'], label='MA20', color='orange')
plt.plot(nvda_data['Date'], nvda_data['MA50'], label='MA50', color='green')
plt.plot(nvda_data['Date'], nvda_data['MA200'], label='MA200', color='red')
plt.title("NVIDIA Closing Price & Moving Averages")
plt.xlabel("Date")
plt.ylabel("Price ($)")
plt.legend()
plt.show()

In [ ]:
#cell2

plt.figure(figsize=(12,5))
sns.histplot(nvda_data['Daily_Return'], bins=100, kde=True, color='purple')
plt.title("Distribution of Daily Returns")
plt.xlabel("Daily Return")
plt.ylabel("Frequency")
plt.show()

In [ ]:
#cell3

plt.figure(figsize=(16,6))
plt.plot(nvda_data['Date'], nvda_data['Volatility'], color='red')
plt.title("20-Day Rolling Volatility of NVIDIA")
plt.xlabel("Date")
plt.ylabel("Volatility")
plt.show()

In [ ]:
#cell4

plt.figure(figsize=(10,6))
sns.heatmap(nvda_data[['Close_NVDA', 'Daily_Return', 'MA20', 'MA50', 'MA200', 'Volatility']].corr(),
            annot=True, cmap='coolwarm')
plt.title("Feature Correlation Heatmap")
plt.show()

#B. CLL

# C. Modélisation
"description partie" 

"base à traduire"
In this section, we prepare features and target for predicting NVIDIA's next-day trading volume.
Lag features, rolling averages, percentage changes, and price range are created to capture trends and volatility.
The dataset is split chronologically into training and testing sets, preserving time-series order,
and TimeSeriesSplit is used for cross-validation to evaluate model performance on future unseen data.
Three regression models are implemented: Decision Tree simple non-linear patterns,
Random Forest ensemble method to reduce overfitting, and XGBoost gradient boosting for complex pattern learning.

## C.1 - Préparations
"explain"

In [ ]:

#  Import Libraries
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
# Lag features
# Lagged values of volume and closing price are created to capture temporal dependencies.
# These features represent how previous days' trading activity affects the current day.
nvda_data['Volume_Lag_1'] = nvda_data['Volume_NVDA'].shift(1)
nvda_data['Volume_Lag_2'] = nvda_data['Volume_NVDA'].shift(2)
nvda_data['Close_Lag_1']  = nvda_data['Close_NVDA'].shift(1)

# Rolling averages
# Short-term moving averages of volume are calculated to smooth fluctuations
# and highlight underlying trends over 3-day and 5-day windows.
nvda_data['Vol_MA_3'] = nvda_data['Volume_NVDA'].rolling(3).mean()
nvda_data['Vol_MA_5'] = nvda_data['Volume_NVDA'].rolling(5).mean()


# Percentage changes
# Compute day-to-day relative changes in volume and closing price.
# These features capture short-term momentum and volatility.
nvda_data['Vol_Change'] = nvda_data['Volume_NVDA'].pct_change()
nvda_data['Close_Change'] = nvda_data['Close_NVDA'].pct_change()

# Price range
# Calculates daily range (High - Low) to capture daily price volatility.
nvda_data['Price_Range'] = nvda_data['High_NVDA'] - nvda_data['Low_NVDA']

# Drop NaNs
# Any rows with missing values (due to lag or rolling calculations) are removed.
# Reset index for clean dataset structure.
nvda_data = nvda_data.dropna().reset_index(drop=True)

# Features & target selection
# X contains selected input features that represent market behavior.
# y is the target variable: the next-day trading volume.
features = [
    'Open_NVDA', 'High_NVDA', 'Low_NVDA', 'Close_NVDA',
    'Volume_Lag_1', 'Volume_Lag_2', 'Close_Lag_1',
    'Vol_MA_3', 'Vol_MA_5', 'Vol_Change', 'Close_Change', 'Price_Range'
]

X = nvda_data[features]
y = nvda_data['Volume_NVDA']


## C.2 - Echantillon entrainement 

In [ ]:

# Train-Test Split
# The dataset is split chronologically: first 80% for training, last 20% for testing.
# This preserves the time-series order and prevents data leakage.
split_index = int(len(X) * 0.8)
X_train = X.iloc[:split_index]
X_test  = X.iloc[split_index:]
y_train = y.iloc[:split_index]
y_test  = y.iloc[split_index:]

# Last row for next-day prediction
# Stores the most recent day’s features to predict the next day’s volume.
last_row = X.iloc[-1].values.reshape(1, -1)

# Time-Series Cross-Validation
# TimeSeriesSplit is used to validate models while respecting temporal order.
# This provides a realistic evaluation of model performance on future unseen data.
tscv = TimeSeriesSplit(n_splits=5)

## C.3 - Decision Tree

### C.3.1 - Modèle

In [ ]:
# Initialize and train the Decision Tree model
# max_depth limits tree depth to prevent overfitting, min_samples_leaf ensures minimum samples per leaf
dt_model = DecisionTreeRegressor(max_depth=5, min_samples_leaf=20, random_state=42)
dt_model.fit(X_train, y_train)

# Predict on test set and next day
# y_pred_dt contains predictions for the test set
# next_day_dt predicts the trading volume for the most recent day
y_pred_dt = dt_model.predict(X_test)
next_day_dt = dt_model.predict(last_row)

# Evaluate model performance
# Metrics include Mean Squared Error (MSE), Mean Absolute Error (MAE), and R² score
# These provide insight into accuracy and how well the model explains the variance in the target
mse_dt = mean_squared_error(y_test, y_pred_dt)
mae_dt = mean_absolute_error(y_test, y_pred_dt)
r2_dt  = r2_score(y_test, y_pred_dt)

### C.3.2 - CCL

In [ ]:
# Display metrics and next-day prediction
print("\nDecision Tree Metrics:")
print("MSE:", mse_dt, "MAE:", mae_dt, "R²:", r2_dt)
print("Next-Day Predicted Volume:", next_day_dt[0])

The R² score shows how much of the overall volume pattern the model correctly predicted.
Example: R² = 0.81 means the model captured 80
% of the trend and variation correctly.
The remaining 19% is due to unpredictable market fluctuations.
This means the model gives a general idea of the trend, but the exact number can vary day to day.

## C.4 Random forest regression
"explain"

This model uses an ensemble of decision trees to predict next day NVIDIA trading volume.
By averaging multiple trees, it reduces overfitting and improves generalization.
Performance is evaluated using MSE, MAE, and R² metrics, and the model forecasts the next day volume.

### C.4.1 - Modèle

In [ ]:
# Initialize and train the Random Forest model
# n_estimators = 300 trees, max_depth limits tree depth, min_samples_leaf ensures minimum samples per leaf
# n_jobs=-1 uses all CPU cores for faster training
rf_model = RandomForestRegressor(n_estimators=300, max_depth=8, min_samples_leaf=20,
                                 random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Predict on test set and next day
# y_pred_rf contains predictions for the test set
# next_day_rf predicts the trading volume for the most recent day
y_pred_rf = rf_model.predict(X_test)
next_day_rf = rf_model.predict(last_row)

# Evaluate model performance
# Metrics include Mean Squared Error (MSE), Mean Absolute Error (MAE), and R² score
# These indicate model accuracy and how well it explains variance in the target
mse_rf = mean_squared_error(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf  = r2_score(y_test, y_pred_rf)

### C.4.2 - CCL

In [ ]:
# Display metrics and next-day prediction
print("\nRandom Forest Metrics:")
print("MSE:", mse_rf, "MAE:", mae_rf, "R²:", r2_rf)
print("Next-Day Predicted Volume:", next_day_rf[0])

Random Forest

The R² score shows how much of the overall volume pattern the model correctly predicted.
Example: R² = 0.94 means the model captured 94% of the trend and variation correctly.
The remaining 6% is due to unpredictable market fluctuations.
This means the model gives a very good estimate of the trend and next-day volume.

## C.5 - XG boost regressio
"explain"
XGBoost is a gradient boosting model that combines weak learners sequentially to predict next day NVIDIA trading volume.
It handles complex patterns in the data, reduces bias, and improves prediction accuracy.
Model performance is evaluated using MSE, MAE, and R², and it is also used to forecast the next day volume.

### C.5.1 - Modèle 

In [ ]:
# Initialize and train the XGBoost model
# n_estimators = 300 trees, max_depth limits tree depth, learning_rate controls step size
# subsample and colsample_bytree control randomness to reduce overfitting
xgb_model = XGBRegressor(n_estimators=300, max_depth=4, learning_rate=0.05,
                         subsample=0.8, colsample_bytree=0.8, random_state=42)
xgb_model.fit(X_train, y_train)

# Predict on test set and next day
# y_pred_xgb contains predictions for the test set
# next_day_xgb predicts the trading volume for the most recent day
y_pred_xgb = xgb_model.predict(X_test)
next_day_xgb = xgb_model.predict(last_row)

# Evaluate model performance
# Metrics include Mean Squared Error (MSE), Mean Absolute Error (MAE), and R² score
# These indicate model accuracy and how well it explains variance in the target
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
r2_xgb  = r2_score(y_test, y_pred_xgb)

### C.5.2 - CCL

In [ ]:

# Display metrics and next-day prediction
print("\nXGBoost Metrics:")
print("MSE:", mse_xgb, "MAE:", mae_xgb, "R²:", r2_xgb)
print("Next-Day Predicted Volume:", next_day_xgb[0])

XGBoost

The R² score shows how much of the overall volume pattern the model correctly predicted.
Example: R² = 0.99 means the model captured 99% of the trend and variation correctly.
Only 1% difference is due to unpredictable market fluctuations.
This means the model gives a very precise estimate of the trend and next-day volume.

# C. CCL

# BILAN FINAL. COMPARAISON ET INTERPRETATION DES RESULTATS

"explain"

In [1]:
plt.figure(figsize=(12,5))

# Plot actual trading volume
plt.plot(y_test.values, label='Actual Volume', color='black', linewidth=2)

# Plot Decision Tree predictions
plt.plot(y_pred_dt, label='Decision Tree', color='blue', alpha=0.7)

# Plot Random Forest predictions
plt.plot(y_pred_rf, label='Random Forest', color='green', alpha=0.7)

# Plot XGBoost predictions
plt.plot(y_pred_xgb, label='XGBoost', color='red', alpha=0.7)

# Chart formatting
plt.title("NVDA Volume Prediction Comparison")
plt.xlabel("Time")
plt.ylabel("Trading Volume")
plt.legend()
plt.grid(True, alpha=0.3)

plt.show()

NameError: name 'plt' is not defined

final ccl 

